In [2]:
import numpy as np
import pandas as pd
from src.data.load import load_dataset
from src.data.process import extract_targets

data = load_dataset()
for k, v in data.items():
    print(k ,len(v))

train 1044681
valid 184937
all_train 1229618
test 178297
all_uid 37960


In [9]:
from src.data.features import extract_user_feature, extract_features

user_features = extract_user_feature(data['all_train'], data['all_uid'])
features_train = extract_features(data['all_train'], user_features)
# features_valid = extract_features(data['valid'], user_features)
features_test = extract_features(data['test'], user_features)
print('feature dim:', features_train.shape[1])
targets_train = extract_targets(data['all_train'], 'log')
# targets_valid = extract_targets(data['valid'], 'linear')

Counting User Histories: 100%|██████████| 1229618/1229618 [00:24<00:00, 50037.92it/s]


feature dim: 34


In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, KFold
import xgboost as xgb
from scipy.stats import randint, uniform
X = features_train
y = targets_train

# 定义K值
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)
# 用于存储每个模型的预测结果
predictions_list = []
# 构建模型
model = xgb.XGBRegressor(max_depth=8, learning_rate=0.09445274248684067, n_estimators=131, min_child_weight=6, n_jobs=-1)

for train_index, test_index in kf.split(features_train):
    # 分割数据
    print('fold index:', train_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 在测试集上进行预测
    predictions = model.predict(features_test)  # 这里使用的是您提到的测试集
    predictions_list.append(predictions)
    
    # 可选：评估模型性能
    val_predictions = model.predict(X_test)
    print("MSE:", mean_squared_error(y_test, val_predictions))
final_predictions = np.mean(predictions_list, axis=0)

fold index: [      0       1       3 ... 1229614 1229616 1229617]
MSE: 2298.3708
fold index: [      2       3       4 ... 1229614 1229615 1229617]
MSE: 5171.099
fold index: [      0       1       2 ... 1229614 1229615 1229616]
MSE: 2068.6677
fold index: [      0       1       2 ... 1229615 1229616 1229617]
MSE: 2524.9993
fold index: [      0       1       2 ... 1229615 1229616 1229617]
MSE: 2450.3135


In [12]:
results = []
for i in range(len(data['test'])):
    results.append(
        "{}\t{}\t{},{},{}\n".format(
            data['test'].loc[i, 'uid'],
            data['test'].loc[i, 'mid'],
            int(final_predictions[i, 1]),
            int(final_predictions[i, 2]),
            int(final_predictions[i, 0])
        )
    )
with open("submission.txt", 'w') as f:
    f.writelines(results)